In [1]:
import pandas as pd
import glob
import time
import duckdb
import sqlalchemy


%load_ext sql

In [2]:
conn = duckdb.connect("mydb.db")

In [3]:
df_user = conn.execute("""
   SELECT * from read_csv_auto('merged_small/users.csv' , header=True)



""").df()
conn.register("df_user_view" , df_user)
conn.execute(" DESCRIBE df_user_view").df()

,column_name,column_type,null,key,default,extra
0,user_id,INTEGER,YES,NaN,NaN,NaN
1,user_name,VARCHAR,YES,NaN,NaN,NaN
2,email,VARCHAR,YES,NaN,NaN,NaN
3,date_of_birth,TIMESTAMP,YES,NaN,NaN,NaN
4,address,VARCHAR,YES,NaN,NaN,NaN
5,state,VARCHAR,YES,NaN,NaN,NaN
6,country,VARCHAR,YES,NaN,NaN,NaN


In [4]:
df_products = conn.execute("""
    SELECT * from read_csv_auto('merged_small/products.csv' , header=True)



""").df()
conn.register("df_products_view" , df_products)
conn.execute(" DESCRIBE df_products_view").df()

,column_name,column_type,null,key,default,extra
0,product_id,VARCHAR,YES,NaN,NaN,NaN
1,product_name,VARCHAR,YES,NaN,NaN,NaN
2,category,VARCHAR,YES,NaN,NaN,NaN
3,price,DOUBLE,YES,NaN,NaN,NaN


In [5]:
df_transactions = conn.execute("""
    SELECT * from read_csv_auto('merged_small/transactions.csv' , header=True)



""").df()
conn.register("df_transactions_view" , df_transactions)
conn.execute(" DESCRIBE df_transactions_view").df()

,column_name,column_type,null,key,default,extra
0,transaction_id,INTEGER,YES,NaN,NaN,NaN
1,user_id,INTEGER,YES,NaN,NaN,NaN
2,product_id,VARCHAR,YES,NaN,NaN,NaN
3,product_name,VARCHAR,YES,NaN,NaN,NaN
4,amount,DOUBLE,YES,NaN,NaN,NaN
5,transaction_date,TIMESTAMP,YES,NaN,NaN,NaN


In [6]:
conn.execute("""
    CREATE OR REPLACE TABLE transactions AS
             SELECT "transaction_id" AS transaction_id,
             "user_id" AS user_id,
             "product_id" as product_id,
             "product_name" as product_name,
             "amount" as amount,
             "transaction_date" as transaction_date
             from df_transactions_view




""")

In [7]:
conn.execute("""
    CREATE OR REPLACE TABLE users AS
             SELECT "user_id" AS user_id,
             "user_name" AS user_name,
             "email" as email,
             "date_of_birth" as date_of_birth,
             "address" as address,
             "state" as state,
             "country" as country
             from df_user_view




""")

In [8]:
conn.execute("""
    CREATE OR REPLACE TABLE products AS
             SELECT "product_id" AS product_id,
             "product_name" AS product_name,
             "category" as category,
             "price" as price
             from df_products_view




""")

BASIC ANALYSIS ON THE TABLES

To view the first 10 rows of all the given tables

In [9]:
conn.execute(" select * from users LIMIT 10").df()

,user_id,user_name,email,date_of_birth,address,state,country
0,1001,Michael Hamilton,jacquelineperry@example.net,1944-10-27,"9550 Smith Ridges\nCynthiastad, MH 80405",Kansas,Mexico
1,1002,William Stokes,cory68@example.org,1970-02-15,"2849 Amanda Bridge Suite 535\nNorth Scott, ME ...",Utah,United Kingdom
2,1003,Tammy Archer,reyesjohn@example.net,1955-03-01,"43205 Hopkins Brook\nCaseyborough, UT 13233",Tennessee,Lithuania
3,1004,Chelsea Herrera,vsingh@example.org,1959-01-01,"800 Emily Harbor Apt. 727\nJonesmouth, FL 51151",Indiana,Romania
4,1005,Peter Thomas,ebentley@example.org,1999-10-12,Unit 5504 Box 8656\nDPO AA 32630,Alaska,Togo
5,1006,Pamela Nelson,fostermichael@example.net,1990-01-23,"0959 Samuel Junction\nPort Natalie, MH 99949",Louisiana,Cuba
6,1007,Eric Nguyen,jhuber@example.com,1968-02-25,"5937 Schneider Heights\nPort Stephenville, IA ...",Arizona,Russian Federation
7,1008,Kimberly Burke,reyesmelissa@example.net,1995-12-15,"646 Daniels Trail Suite 903\nThomashaven, FL 6...",Nebraska,Swaziland
8,1009,Steven Blake,ronald02@example.org,1990-02-11,"8619 Michael Divide\nFoxstad, CO 16596",Oklahoma,Qatar
9,1010,Mackenzie Fitzgerald,chadmiller@example.net,1964-03-28,"412 Carter Union Apt. 761\nStacymouth, ND 44214",Montana,Belarus


In [10]:
conn.execute(" select * from products LIMIT 10").df()

,product_id,product_name,category,price
0,P689,blood,Clothing,175.56
1,P802,cultural,Accessories,85.98
2,P821,turn,Clothing,115.44
3,P977,notice,Tools,27.76
4,P403,including,Clothing,55.68
5,P322,will,Tools,29.27
6,P221,push,Tools,76.42
7,P728,anyone,Electronics,35.98
8,P469,everything,Tools,153.82
9,P138,tree,Home,147.97


In [11]:
conn.execute(" select * from transactions LIMIT 10").df()

,transaction_id,user_id,product_id,product_name,amount,transaction_date
0,1,1100,P148,job,50.03,2023-04-27 03:57:05
1,2,1195,P937,never,48.00,2023-05-14 22:55:01
2,3,1934,P244,threat,39.08,2023-06-04 21:53:53
3,4,1081,P113,art,96.16,2023-06-14 10:53:56
4,5,1399,P981,foot,35.09,2023-05-13 05:44:19
5,6,1452,P803,center,77.23,2023-06-27 17:31:51
6,7,1559,P938,doctor,18.84,2023-04-25 23:59:38
7,8,1701,P384,also,96.38,2023-02-19 08:58:17
8,9,1024,P918,control,44.15,2023-07-20 16:28:37
9,10,1822,P112,realize,36.15,2023-06-03 11:56:52


Count of total number of rows in each Table

In [12]:
conn.execute("  SELECT count(*) AS user_count FROM users  ").df()



,user_count
0,5100000


In [13]:
conn.execute("  SELECT count(*) AS product_count FROM products  ").df()

,product_count
0,300000


In [14]:
conn.execute("  SELECT count(*) AS transaction_count FROM transactions  ").df()

,transaction_count
0,10100000


Statestical data on the Tables

In [15]:
conn.execute("""
             
             SELECT MIN(amount) , MAX(amount) , AVG(amount) FROM transactions
             
             
             """).df()

,min(amount),max(amount),avg(amount)
0,10.0,100.0,55.000823


Missing Values

In [16]:

df_user.isnull().sum()

user_id          0
user_name        0
email            0
date_of_birth    0
address          0
state            0
country          0
dtype: int64

In [17]:
df_products.isnull().sum()

product_id      0
product_name    0
category        0
price           0
dtype: int64

In [18]:
df_transactions.isnull().sum()

transaction_id      0
user_id             0
product_id          0
product_name        0
amount              0
transaction_date    0
dtype: int64

There is no null values in any of the tables (expected as the data was synthetically created) so no need to handle any null values

Data Cleaning Procedure:
    Check for duplicate values

In [19]:
conn.execute("SELECT transaction_id, user_id, product_id, transaction_date, COUNT(*) AS count FROM transactions GROUP BY transaction_id, user_id, product_id, transaction_date HAVING COUNT(*) > 1").df()

,transaction_id,user_id,product_id,transaction_date,count


In [20]:
conn.execute("SELECT user_id, user_name, email, date_of_birth, address, state, country, COUNT(*) AS count FROM users GROUP BY user_id, user_name, email, date_of_birth, address, state, country HAVING COUNT(*) > 1").df()

,user_id,user_name,email,date_of_birth,address,state,country,count


In [21]:
conn.execute("SELECT product_id, product_name, category, price, COUNT(*) AS count FROM products GROUP BY product_id, product_name, category, price HAVING COUNT(*) > 1").df()

,product_id,product_name,category,price,count
0,P381,guess,Tools,141.11,2
1,P121,ability,Tools,60.61,2
2,P350,agent,Electronics,178.71,2


In [22]:
conn.execute("SELECT product_id, product_name, category, price, COUNT(*) AS count FROM products GROUP BY product_id, product_name, category, price HAVING COUNT(*) > 1").df()

,product_id,product_name,category,price,count
0,P381,guess,Tools,141.11,2
1,P121,ability,Tools,60.61,2
2,P350,agent,Electronics,178.71,2


Now removing the duplicates from the products table

In [23]:
conn.execute("""
    delete from products where "product_id" == 'P381' OR "product_id" == 'P121' OR "product_id" == 'P350'



""")

In [24]:
conn.execute("SELECT product_id, product_name, category, price, COUNT(*) AS count FROM products GROUP BY product_id, product_name, category, price HAVING COUNT(*) > 1").df()

,product_id,product_name,category,price,count


Performing complex aggregated qerries

1. Calculating total revenue by category

In [25]:
conn.execute("""
    SELECT p.category, SUM(t.amount) AS total_revenue
    FROM transactions t
    JOIN products p ON t.product_id = p.product_id
    GROUP BY p.category
""").df()







,category,total_revenue
0,Tools,3.692172e+10
1,Home,3.713058e+10
2,Electronics,3.684414e+10
3,Clothing,3.679069e+10
4,Accessories,3.684177e+10


2. Finding Users with the highest total spending by country

In [26]:
conn.execute("""
    SELECT u.country, u.user_id, u.user_name, SUM(t.amount) AS total_spending
    FROM transactions t
    JOIN df_user_view u ON t.user_id = u.user_id
    GROUP BY u.country, u.user_id, u.user_name
    ORDER BY total_spending DESC
""").df()

,country,user_id,user_name,total_spending
0,North Macedonia,1846,Sheila Walker,581900.17
1,Belarus,1216,Richard Brown,576344.03
2,Sri Lanka,1903,Peter Wilkerson,572889.68
3,Guinea-Bissau,1558,Adam Morgan,571958.33
4,Kenya,1183,Nancy Cowan,571649.80
...,...,...,...,...
995,Spain,1575,Christine Newton,539349.18
996,Turks and Caicos Islands,1904,Claire Williams,539337.86
997,Swaziland,1359,Michael Evans,538964.12
998,Gambia,1547,Katherine Beard,538155.30


3. Calculating the average Transaction amount per day

In [27]:
conn.execute("""
    SELECT transaction_date AS transaction_day, AVG(amount) AS avg_transaction_amount
    FROM transactions
    GROUP BY transaction_date
""").df()

,transaction_day,avg_transaction_amount
0,2023-06-30 03:55:36,89.440
1,2023-06-26 09:28:19,98.820
2,2023-01-12 11:55:31,51.065
3,2023-05-24 04:31:00,33.255
4,2023-07-25 12:43:00,44.650
...,...,...
7763304,2023-01-26 15:11:40,22.850
7763305,2023-07-31 09:17:33,59.200
7763306,2023-07-23 07:07:37,28.560
7763307,2023-03-21 14:01:27,66.650


4. Top Selling Product categories wuth average price 


In [28]:
conn.execute("""
    SELECT p.category, COUNT(*) AS num_transactions, AVG(p.price) AS avg_price
    FROM transactions t
    JOIN products p ON t.product_id = p.product_id
    GROUP BY p.category
    ORDER BY num_transactions DESC
""").df()

,category,num_transactions,avg_price
0,Home,675102657,105.348418
1,Tools,671317535,105.086167
2,Electronics,669890742,105.035111
3,Accessories,669854502,104.949909
4,Clothing,668918068,104.937401


5. Grouping users by age and displaying it in ascending order of age 

In [29]:
conn.execute("""
    SELECT EXTRACT(YEAR FROM NOW()) - EXTRACT(YEAR FROM date_of_birth) AS age, COUNT(*) AS user_count
    FROM users
    GROUP BY age
    ORDER BY age
""").df()

,age,user_count
0,18,46641
1,19,80726
2,20,81252
3,21,81244
4,22,80591
...,...,...
59,77,80500
60,78,80909
61,79,80847
62,80,81287


Optimizing the query 

We can see that querry  1. Calculating total revenue by category and 4. Top Selling Product categories wuth average price took substantiall longer time to execte. Lets try optimizing the query by appling certain optimization techniques


1. Applying indexing

In [30]:
# indexing  "product_id" column in the "transactions" table
conn.execute("  DROP INDEX IF EXISTS idx_transactions_product_id ")
conn.execute("CREATE INDEX idx_transactions_product_id ON transactions(product_id)")

# indexing the "category" in products
conn.execute("  DROP INDEX IF EXISTS idx_products_category ")
conn.execute("CREATE INDEX idx_products_category ON products(category)")


In [33]:
conn.execute("""
    SELECT p.category, SUM(t.amount) AS total_revenue
    FROM transactions t
    JOIN products p ON t.product_id = p.product_id
    GROUP BY p.category
""").df()

,category,total_revenue
0,Clothing,3.679069e+10
1,Tools,3.692172e+10
2,Electronics,3.684414e+10
3,Accessories,3.684177e+10
4,Home,3.713058e+10


we see the exection time reduce from 40.2 to 35.7 seconds. Its a 11.2% reduction

In [34]:
conn.execute("""
    SELECT p.category, COUNT(*) AS num_transactions, AVG(p.price) AS avg_price
    FROM transactions t
    JOIN products p ON t.product_id = p.product_id
    GROUP BY p.category
    ORDER BY num_transactions DESC
""").df()

,category,num_transactions,avg_price
0,Home,675102657,105.348418
1,Tools,671317535,105.086167
2,Electronics,669890742,105.035111
3,Accessories,669854502,104.949909
4,Clothing,668918068,104.937401


We can see that the execution time reduced from 41.1 sec to 39.9 sec, That is a 2.9 percent reduction in running time